In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer

import os
from tqdm import tqdm
tqdm.pandas()

import re
# from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

seed = 42
np.random.seed(seed)

In [ ]:
# input
csv_dir = "/content/drive/MyDrive/Colab Notebooks/Drain_result/"
# dataset_prefix = "BGL."
dataset_prefix = "AIT."
csv_structured = csv_dir + dataset_prefix + "log_ident_structured.csv"
csv_template = csv_dir + dataset_prefix + "log_ident_templates.csv"

# test_portion = 0.6
# cv = 3

In [ ]:
struct_log = {
    "spiral.com": {},
    "onion.com": {},
    "insect.com": {},
    "cup.com": {}
}

label_log = {
    "spiral.com": {},
    "onion.com": {},
    "insect.com": {},
    "cup.com": {}
}

template_log = {
    "spiral.com": {},
    "onion.com": {},
    "insect.com": {},
    "cup.com": {}
}

In [ ]:
def str_to_list(x):
  if x == "[]": return []
  x = x.lstrip('[').rstrip(']')
  X = x.split(', ')
  return [ p.strip("'") for p in X ]

In [ ]:
def define_dtype(df, dtype_map, copy=False):
  if copy: df = df.copy()
  
  for col, dtype in dtype_map.items():
    if col not in df.columns: continue
    if dtype.startswith("datetime_format_"):
      dtype, format = dtype.split("_format_")
      df[col] = pd.to_datetime(df[col], format=format)
    if dtype == "list":
      df[col] = df[col].map(str_to_list)
    elif dtype == "_drop_": df = df.drop(columns=[col])
    else: df[col] = df[col].astype(dtype)
  
  return df

In [ ]:
for (root,dirs,files) in os.walk(f"{csv_dir}", topdown=True):
  if root == csv_dir: continue
  if len(files) > 0:
    for f in files:
      if f.endswith(".log"):
        with open(f'{root}/{f}', 'r') as original: 
          data = original.read()
          with open(f'{root}/{f}.label', 'w') as modified:
            if not data.startswith("time_label,line_label"):
              print(f"{root}/{f}")
              modified.write("time_label,line_label\n" + data)
            else: modified.write(data)

/content/drive/MyDrive/Colab Notebooks/Drain_result/AIT_auth_mail.spiral.com/auth.log
/content/drive/MyDrive/Colab Notebooks/Drain_result/AIT_daemon_mail.onion.com/daemon.log
/content/drive/MyDrive/Colab Notebooks/Drain_result/AIT_daemon_mail.spiral.com/daemon.log
/content/drive/MyDrive/Colab Notebooks/Drain_result/AIT_auth_mail.onion.com/auth.log
/content/drive/MyDrive/Colab Notebooks/Drain_result/AIT_mail_mail.insect.com/mail.log
/content/drive/MyDrive/Colab Notebooks/Drain_result/AIT_daemon_mail.insect.com/daemon.log
/content/drive/MyDrive/Colab Notebooks/Drain_result/apache2_mail.onion.com/mail.onion.com-access.log
/content/drive/MyDrive/Colab Notebooks/Drain_result/AIT_mail_mail.spiral.com/mail.log
/content/drive/MyDrive/Colab Notebooks/Drain_result/AIT_user_mail.onion.com/user.log
/content/drive/MyDrive/Colab Notebooks/Drain_result/apache2_mail.insect.com/mail.insect.com-access.log
/content/drive/MyDrive/Colab Notebooks/Drain_result/AIT_user_mail.spiral.com/user.log
/content/driv

In [ ]:
for (root,dirs,files) in os.walk(f"{csv_dir}", topdown=True):
  if root == csv_dir: continue
  print(f"{root}")
  if len(files) > 0:
    for f in files:
      if f.endswith(".label"):
        with open(f'{root}/{f}', 'r') as original: 
          data = original.readline()
          print(f"{f}: {data}")

/content/drive/MyDrive/Colab Notebooks/Drain_result/AIT_auth_mail.spiral.com
auth.log.label: time_label,line_label

/content/drive/MyDrive/Colab Notebooks/Drain_result/AIT_daemon_mail.onion.com
daemon.log.label: time_label,line_label

/content/drive/MyDrive/Colab Notebooks/Drain_result/AIT_daemon_mail.spiral.com
daemon.log.label: time_label,line_label

/content/drive/MyDrive/Colab Notebooks/Drain_result/AIT_auth_mail.onion.com
auth.log.label: time_label,line_label

/content/drive/MyDrive/Colab Notebooks/Drain_result/AIT_auth_mail.insect.com
auth.log.label: time_label,line_label

/content/drive/MyDrive/Colab Notebooks/Drain_result/AIT_auth_mail.cup.com
auth.log.label: time_label,line_label

/content/drive/MyDrive/Colab Notebooks/Drain_result/AIT_daemon_mail.cup.com
daemon.log.label: time_label,line_label

/content/drive/MyDrive/Colab Notebooks/Drain_result/AIT_mail_mail.cup.com
mail.log.label: time_label,line_label

/content/drive/MyDrive/Colab Notebooks/Drain_result/AIT_mail_mail.insec

In [ ]:
for (root,dirs,files) in os.walk(f"{csv_dir}", topdown=True):
  if root == csv_dir: continue

  print(f"root: {root}")
  if len(dirs) > 0:
    print("dirs:")
    for d in dirs:
      print(f"- {d}")
  if len(files) > 0:
    print("files:")
    for f in files:
      print(f"- {f}")
      _dot_indx = root.rfind(".", 0, -4)
      keys = root[_dot_indx + 1:]
      if f.endswith("log_ident_structured.csv"):
        struct_log[keys][f[:-25]] = pd.read_csv(f"{root}/{f}")
      if f.endswith("log_ident_templates.csv"):
        template_log[keys][f[:-24]] = pd.read_csv(f"{root}/{f}")
      if f.endswith(".log.label"):
        label_log[keys][f[:-10]] = pd.read_csv(f"{root}/{f}")
  print('--------------------------------')

root: /content/drive/MyDrive/Colab Notebooks/Drain_result/AIT_auth_mail.spiral.com
files:
- auth.log_ident_templates.csv
- auth.log_ident_structured.csv
- auth.log
- auth.log.label
--------------------------------
root: /content/drive/MyDrive/Colab Notebooks/Drain_result/AIT_daemon_mail.onion.com
files:
- daemon.log_ident_templates.csv
- daemon.log_ident_structured.csv
- daemon.log
- daemon.log.label
--------------------------------
root: /content/drive/MyDrive/Colab Notebooks/Drain_result/AIT_daemon_mail.spiral.com
files:
- daemon.log_ident_templates.csv
- daemon.log_ident_structured.csv
- daemon.log
- daemon.log.label
--------------------------------
root: /content/drive/MyDrive/Colab Notebooks/Drain_result/AIT_auth_mail.onion.com
files:
- auth.log_ident_structured.csv
- auth.log_ident_templates.csv
- auth.log
- auth.log.label
--------------------------------
root: /content/drive/MyDrive/Colab Notebooks/Drain_result/AIT_auth_mail.insect.com
files:
- auth.log_ident_templates.csv
- aut

In [ ]:
for w in struct_log:
  for k in struct_log[w]:
    if len(label_log[w][k]) != len(struct_log[w][k]):
      print(f"error: ({w}, {k}) unmatch length, got ({len(label_log[w][k])}, {len(struct_log[w][k])})")
    else: struct_log[w][k] = pd.concat([label_log[w][k], struct_log[w][k]], axis=1)

In [ ]:
struct_log["spiral.com"]["mail.spiral.com-access"]["time_label"].unique()

array(['0', 'nikto', 'vrfy', 'hydra', 'upload', 'mail-curl',
       'webshell-lsbrelease-a', 'webshell-ps-aux', 'webshell-netstat-u',
       'webshell-id', 'webshell-ls-home', 'webshell-cd',
       'webshell-cd-etc', 'webshell-ps-A', 'webshell-netstat-nau',
       'webshell-dpkg-exim4', 'webshell-chmod-eximsploitsh',
       'webshell-netstat-t', 'webshell-ls-www', 'webshell-find-null',
       'webshell-cat-passwd', 'webshell-ifconfig'], dtype=object)

In [ ]:
struct_log.keys()

dict_keys(['spiral.com', 'onion.com', 'insect.com', 'cup.com'])

In [ ]:
for k in struct_log:
  print(k, ":", struct_log[k].keys())

spiral.com : dict_keys(['auth', 'daemon', 'mail', 'user', 'mail.spiral.com-access'])
onion.com : dict_keys(['daemon', 'auth', 'mail.onion.com-access', 'user', 'mail'])
insect.com : dict_keys(['auth', 'mail', 'daemon', 'mail.insect.com-access', 'user'])
cup.com : dict_keys(['auth', 'daemon', 'mail', 'user', 'mail.cup.com-access'])


In [ ]:
struct_log["onion.com"]["mail.onion.com-access"].sample(5)

,time_label,line_label,LineId,IP,DateTime,Content,EventId,EventTemplate,EventTemplateIdent,ParameterList
10329,0,0,10330,192.168.10.185,01/Mar/2020:16:07:22 +0000,GET /turba/themes/default/graphics/turba.png H...,1b4fe6f6,"<*> <*> HTTP/<*>.<*>"" <*> <*> <*> ""Mozilla/<*>...",<Method> <similar_prev_pattern> HTTP/<Numbers>...,"['GET', '/turba/themes/default/graphics/turba...."
7082,0,0,7083,192.168.10.185,01/Mar/2020:11:22:46 +0000,GET /imp/dynamic.php?page=compose&type=new&tok...,1b4fe6f6,"<*> <*> HTTP/<*>.<*>"" <*> <*> <*> ""Mozilla/<*>...",<Method> <similar_prev_pattern> HTTP/<Numbers>...,"['GET', '/imp/dynamic.php?page=compose&type=ne..."
46856,0,0,46857,192.168.10.185,04/Mar/2020:11:34:54 +0000,"POST /nag/task/save.php HTTP/1.1"" 302 414 ""htt...",1b4fe6f6,"<*> <*> HTTP/<*>.<*>"" <*> <*> <*> ""Mozilla/<*>...",<Method> <similar_prev_pattern> HTTP/<Numbers>...,"['POST', '/nag/task/save.php', '1', '1', '302'..."
45795,0,0,45796,192.168.10.185,04/Mar/2020:09:52:26 +0000,"GET /kronolith/ HTTP/1.1"" 200 24856 ""http://ma...",1b4fe6f6,"<*> <*> HTTP/<*>.<*>"" <*> <*> <*> ""Mozilla/<*>...",<Method> <similar_prev_pattern> HTTP/<Numbers>...,"['GET', '/kronolith/', '1', '1', '200', '24856..."
19570,0,0,19571,192.168.10.185,02/Mar/2020:10:40:57 +0000,POST /services/ajax.php/kronolith/listTopTags ...,1b4fe6f6,"<*> <*> HTTP/<*>.<*>"" <*> <*> <*> ""Mozilla/<*>...",<Method> <similar_prev_pattern> HTTP/<Numbers>...,"['POST', '/services/ajax.php/kronolith/listTop..."


In [ ]:
for w in struct_log:
  for k in struct_log[w]:
    print(f"({w}) {k}", end=":\t")
    for c in struct_log[w][k].columns:
      print(c[:7], end="\t")
    print()

(spiral.com) auth:	time_la	line_la	LineId	Month	Day	Time	Type	Job	Pam_uni	Content	EventId	EventTe	EventTe	Paramet	
(spiral.com) daemon:	time_la	line_la	LineId	Month	Day	Time	Type	Job	Content	EventId	EventTe	EventTe	Paramet	
(spiral.com) mail:	time_la	line_la	LineId	Month	Day	Time	Type	Job	Access	Content	EventId	EventTe	EventTe	Paramet	
(spiral.com) user:	time_la	line_la	LineId	Month	Day	Time	Type	Job	Content	EventId	EventTe	EventTe	Paramet	
(spiral.com) mail.spiral.com-access:	time_la	line_la	LineId	IP	DateTim	Content	EventId	EventTe	EventTe	Paramet	
(onion.com) daemon:	time_la	line_la	LineId	Month	Day	Time	Type	Job	Content	EventId	EventTe	EventTe	Paramet	
(onion.com) auth:	time_la	line_la	LineId	Month	Day	Time	Type	Job	Pam_uni	Content	EventId	EventTe	EventTe	Paramet	
(onion.com) mail.onion.com-access:	time_la	line_la	LineId	IP	DateTim	Content	EventId	EventTe	EventTe	Paramet	
(onion.com) user:	time_la	line_la	LineId	Month	Day	Time	Type	Job	Content	EventId	EventTe	EventTe	Paramet	
(onio

In [ ]:
# struct_log = pd.read_csv(csv_structured)
# template_log = pd.read_csv(csv_template)

In [ ]:
struct_dtype = {
    "time_label": "string",
    "line_label": "string",
    "LineId": "int64",
    "Month": "string",
    "Day": "int32",
    "Time": "string",
    "Type": "string",
    "Job": "string",
    "Pam_unix": "string",
    "Access": "string",
    "IP": "string",
    "DateTime": "string", #02/Mar/2020:22:00:18 +0000
    "Content": "string",
    "EventId": "string",
    "EventTemplate": "string",
    "EventTemplateIdent": "string",
    "ParameterList": "list",
}

template_dtype = {
    "EventId": "string",
    "EventTemplateIdent": "string",
    "Occurrences": "int64"
}

In [ ]:
# struct_log = define_dtype(struct_log, struct_dtype)
# template_log = define_dtype(template_log, template_dtype)

In [ ]:
for w in struct_log:
  for k in struct_log[w]:
    struct_log[w][k] = define_dtype(struct_log[w][k], struct_dtype)
    template_log[w][k] = define_dtype(template_log[w][k], template_dtype)

In [ ]:
for w in struct_log:
  for k in struct_log[w]:
    struct_log[w][k]["time_label"] = struct_log[w][k]["time_label"].map(lambda x: int(x != '0'))
    struct_log[w][k]["line_label"] = struct_log[w][k]["line_label"].map(lambda x: int(x != '0'))

In [ ]:
# struct_log.info()

In [ ]:
for w in struct_log: print(w, ":", ", ".join([ k for k in struct_log[w] ]))

spiral.com : auth, daemon, mail, user, mail.spiral.com-access
onion.com : daemon, auth, mail.onion.com-access, user, mail
insect.com : auth, mail, daemon, mail.insect.com-access, user
cup.com : auth, daemon, mail, user, mail.cup.com-access


In [ ]:
df = struct_log["insect.com"]["mail.insect.com-access"]
df[df["EventTemplateIdent"].str.contains("<similar_prev_pattern")]\
  [["Content", "EventId", "EventTemplateIdent", "ParameterList"]].groupby("EventTemplateIdent").first().reset_index()

,EventTemplateIdent,Content,EventId,ParameterList
0,<Method> <similar_prev_pattern> HTTP/<Numbers>...,"GET /./ HTTP/1.0"" 302 553 ""-"" ""Mozilla/5.00 (N...",bcdff403,"[GET, /./, 1, 0, 302, 553, 5, 00, 2, 1, 5, 2, ..."
1,<Method> <similar_prev_pattern> HTTP/<Numbers>...,"GET /./cgi.cgi/./ HTTP/1.1"" 302 658 ""-"" ""Mozil...",8827c04d,"[GET, /./cgi.cgi/./, 1, 1, 302, 658, 5, 00, 2,..."
2,<Method> <similar_prev_pattern> HTTP/<Numbers>...,"GET /login.php HTTP/1.1"" 200 6312 ""-"" ""python-...",0ffeac22,"[GET, /login.php, 1, 1, 200, 6312, ""python-req..."
3,<Method> <similar_prev_pattern> HTTP/<Numbers>...,"GET /login.php HTTP/1.1"" 200 2538 ""-"" ""Mozilla...",1b4fe6f6,"[GET, /login.php, 1, 1, 200, 2538, ""-"", 5, 0, ..."
4,<Method> <similar_prev_pattern> HTTP/<Numbers>...,"GET /login.php HTTP/1.1"" 200 2538 ""-"" ""Mozilla...",2c654ef4,"[GET, /login.php, 1, 1, 200, 2538, ""-"", 5, 0, ..."
5,<similar_prev_pattern> <similar_prev_pattern> ...,"GET /./site/./' UNION"" 400 0 ""-"" ""-",7a7cd532,"[GET, ""/./site/./'"", UNION"", 400, 0]"
6,<similar_prev_pattern> <similar_prev_pattern> ...,"GET /./ HTTP/1.1"" 302 553 ""-"" ""Mozilla/5.00 (N...",ce816de1,"[GET, /./, 1, 1, 302, 553, 5, 00, 2, 1, 5, 2, ..."
7,<similar_prev_pattern> <similar_prev_pattern> ...,"HEAD /./ HTTP/1.1"" 302 512 ""-"" ""Mozilla/5.00 (...",db4f1246,"[HEAD, /./, 1, 1, 302, 512, 5, 00, 2, 1, 5, 2,..."


In [ ]:
def encode_param_by_tag(x, tags=["<similar_prev_pattern>"]):
  return [ any([t == tag for tag in tags]) for t in re.findall("<[\w_]+>", x)]

In [ ]:
regexes = [
    # ("<Hexadecimal>", r"(0[xX])[a-fA-F0-9]{0,15}[a-fA-F0-9]"),
    # ("<Hexadecimal>", r"[A-Fa-f0-9]{16}"),
    # ("<Hexadecimal>", r"[A-Fa-f0-9]{13}"),
    # ("<Hexadecimal>", r"[A-Fa-f0-9]{11}"),
    # ("<Hexadecimal>", r"[A-Fa-f0-9]{8}"),
    # ("<Number>:<Hexadecimal>", r"[0-9]{1,2}:[A-Fa-f0-9]{8}"),
    # ("<Number>,", r"[0-9]+,"),
    # ("<Core>", r"core.+"),
    # ("<IP>", r"(/|)([0-9]+\.){3}[0-9]+(:[0-9]+|)(:|)"),
    # ("<Path>", r"(\/.*?\/)((?:[^\/]|\\\/)+?)(?:(?<!\\):|$)"),
    # ("<Numbers>", r"(?<=[^A-Za-z0-9])(\-?\+?\d+)(?=[^A-Za-z0-9])|[0-9]+$"),
    # ("<Node>", r"[RJ][0-9]{1,2}(-M[0-9])?-[A-Z0-9]+[,]?"),
    # ("<Identifier>", r"[A-F0-9]{2}(:[A-F0-9]{2}){11}"),
    ("<Mail>", r"<?[a-zA-Z0-9._]+@[a-zA-Z0-9.]>?[,]?"),
    ("<Path>", r'"(\/[\w.@_\:]*)+"\]?'),
    ("<Path>", r'(\/[\w.@_\:]*)+'),
    ("<URL>", r'"http:/(\/[\w.@_\=&\?]+)+"'),
    ("<Second>", r"[0-9]+.[0-9]{6}s"),
    ("<Function>", r"[a-zA-F_0-9]+\([\w\/.]+\)"),
    ("__=<Identifier>", r"[a-z]+=[A-Za-z0-9]+[,]?"),
]

In [ ]:
def labeling(param):
  global regexes
  for tag, regex in regexes:
    # print(tag, regex)
    if tag.startswith("__"):
      sym_i = param.find(tag[2])
      return param[:sym_i + 1] + tag[3:]
    else:
      y = re.fullmatch(regex, param)
      if y is not None: return tag
  
  # if re.fullmatch("[a-z]+=[0-9]+[,]?", param):
  #   eq_i = param.find("=")
  #   return param[:eq_i + 1] + "<Number>"
    
  return param

In [ ]:
for w in struct_log:
  for k in struct_log[w]:
    df = struct_log[w][k].copy()
    cols = df.columns.to_list()
    for rc in ["EventId", "EventTemplateIdent"]: cols.remove(rc)

    params_is_spp_tag = df["EventTemplateIdent"].progress_apply(encode_param_by_tag, tags=["<similar_prev_pattern>", "<Method>"])
    et = list()
    for i, row in tqdm(enumerate(df[["EventTemplateIdent", "ParameterList"]].values), total=len(df)):
      if len(row[1]) <= 0: 
        et.append(row[0])
      else:
        try: l = [ labeling(row[1][j]) for j, tag in enumerate(params_is_spp_tag[i]) if tag ]
        except:
          print(f"\n{params_is_spp_tag[i]}")
          print(row[1])
          break
        if len(l) <= 0:
          et.append(row[0])
        else:
          y = row[0]
          for t in l: y = y.replace("<similar_prev_pattern>", t, 1)
          et.append(y)

    df["EventTemplateIdent-2"] = et
    eid = [f"{w}-{k}" + f"000{i}"[-3:] for i in range(len(df["EventTemplateIdent-2"].unique()))]
    template_log[w][k] = pd.DataFrame({
        "EventId": eid,
        "EventTemplateIdent": df["EventTemplateIdent-2"].unique()
    })
    tmp = df[cols]
    tmp["EventTemplateIdent"] = df["EventTemplateIdent-2"]
    temp_dict = template_log[w][k].set_index("EventTemplateIdent").to_dict()["EventId"]
    new_eid = [ temp_dict[t] for t in tmp["EventTemplateIdent"] ]
    tmp["EventId"] = new_eid
    struct_log[w][k] = tmp

100%|██████████| 1202/1202 [00:00<00:00, 125162.70it/s]
<ipython-input-29-3ec2fd09da1d>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp["EventTemplateIdent"] = df["EventTemplateIdent-2"]
<ipython-input-29-3ec2fd09da1d>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp["EventId"] = new_eid
100%|██████████| 905/905 [00:00<00:00, 329071.97it/s]
<ipython-input-29-3ec2fd09da1d>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [ ]:
output_dir = "/content/drive/MyDrive/Colab Notebooks/Parsed_Log/AIT/"

for w in struct_log:
  for k in struct_log[w]:
    struct_log_path = f"{output_dir}{dataset_prefix}{w}-{k}.log_ident_structured.pkl"
    template_log_path = f"{output_dir}{dataset_prefix}{w}-{k}.log_ident_templates.pkl"

    struct_log[w][k].to_pickle(struct_log_path)
    template_log[w][k].to_pickle(template_log_path)
    
    print("saving log structured:", struct_log_path)
    print("saving log templates :", template_log_path)

saving log structured: /content/drive/MyDrive/Colab Notebooks/Parsed_Log/AIT/AIT.spiral.com-auth.log_ident_structured.pkl
saving log templates : /content/drive/MyDrive/Colab Notebooks/Parsed_Log/AIT/AIT.spiral.com-auth.log_ident_templates.pkl
saving log structured: /content/drive/MyDrive/Colab Notebooks/Parsed_Log/AIT/AIT.spiral.com-daemon.log_ident_structured.pkl
saving log templates : /content/drive/MyDrive/Colab Notebooks/Parsed_Log/AIT/AIT.spiral.com-daemon.log_ident_templates.pkl
saving log structured: /content/drive/MyDrive/Colab Notebooks/Parsed_Log/AIT/AIT.spiral.com-mail.log_ident_structured.pkl
saving log templates : /content/drive/MyDrive/Colab Notebooks/Parsed_Log/AIT/AIT.spiral.com-mail.log_ident_templates.pkl
saving log structured: /content/drive/MyDrive/Colab Notebooks/Parsed_Log/AIT/AIT.spiral.com-user.log_ident_structured.pkl
saving log templates : /content/drive/MyDrive/Colab Notebooks/Parsed_Log/AIT/AIT.spiral.com-user.log_ident_templates.pkl
saving log structured: /

In [ ]:
for w in struct_log:
  print(w)

spiral.com
onion.com
insect.com
cup.com


In [ ]:
# output_dir = "/content/drive/MyDrive/Colab Notebooks/Parsed_Log/"

# pkl_train_path = list()
# pkl_test_path = list()
# pkl_prefix = output_dir + dataset_prefix

# for i in range(cv):
#   # trainset
#   pkl_train_path.append({
#       "structured"    : f"{pkl_prefix}cv{i}_train.log_ident_structured.pkl",
#       "template"      : f"{pkl_prefix}cv{i}_train.log_ident_templates.pkl",
#   })
#   # testset
#   pkl_test_path.append({
#       "structured"    : f"{pkl_prefix}cv{i}_test.log_ident_structured.pkl",
#       "template"      : f"{pkl_prefix}cv{i}_test.log_ident_templates.pkl",
#   })

In [ ]:
# kf = KFold(n_splits=cv, shuffle=False)
# for i, (train_index, test_index) in enumerate(kf.split(struct_log)):
#   print(f"{i + 1}/{cv} fold:")
#   trainset = struct_log.iloc[train_index].reset_index(drop=True)
#   testset =  struct_log.iloc[test_index].reset_index(drop=True)

#   train_eventId = trainset["EventId"]
#   train_template = template_log.query("EventId in @train_eventId")

#   test_eventId = testset["EventId"]
#   test_template = template_log.query("EventId in @test_eventId")

#   trainset.to_pickle(pkl_train_path[i]["structured"])
#   testset.to_pickle(pkl_test_path[i]["structured"])
#   print("trainset:", pkl_train_path[i]["structured"])
#   print("testset:", pkl_test_path[i]["structured"])

#   train_template.to_pickle(pkl_train_path[i]["template"])
#   test_template.to_pickle(pkl_test_path[i]["template"])

1/3 fold:
trainset: /content/drive/MyDrive/Colab Notebooks/Parsed_Log/BGL.cv0_train.log_ident_structured.pkl
testset: /content/drive/MyDrive/Colab Notebooks/Parsed_Log/BGL.cv0_test.log_ident_structured.pkl
2/3 fold:
trainset: /content/drive/MyDrive/Colab Notebooks/Parsed_Log/BGL.cv1_train.log_ident_structured.pkl
testset: /content/drive/MyDrive/Colab Notebooks/Parsed_Log/BGL.cv1_test.log_ident_structured.pkl
3/3 fold:
trainset: /content/drive/MyDrive/Colab Notebooks/Parsed_Log/BGL.cv2_train.log_ident_structured.pkl
testset: /content/drive/MyDrive/Colab Notebooks/Parsed_Log/BGL.cv2_test.log_ident_structured.pkl
